In [1]:
import numpy as np
import matplotlib as plt

## Spectral analysis with numpy
Numpy has a submodule for the spectral analysis.

In [2]:
from numpy import linalg as la

#### Get the adjacency matrix of the Florentine families

![](../Images/ff_network.png)

In [3]:
with open('../Data/florentine_families.dat', 'r') as file:
    all_file=file.readlines()

In [4]:
families=np.array(all_file[4:20])

In [5]:
ff_adj=np.genfromtxt('../Data/florentine_families.dat', skip_header=41, dtype='i8')[:16]

In [6]:
ff_adj.shape

(16, 16)

In [7]:
np.all(ff_adj.T==ff_adj)

True

In [8]:
del file, all_file

### Eigenvalues and eigenvectors

For real symmetric or Hermitian matrices use eigh. The first output is the array of eigenvalues (sorted), the second is an array of eigenvectors, in the same order.

In [9]:
la.eigh(ff_adj)

(array([-2.69583872e+00, -2.06786891e+00, -1.87072241e+00, -1.19329397e+00,
        -8.69346719e-01, -7.65384963e-01, -5.76260635e-01, -2.02434814e-01,
        -6.53638385e-17,  2.57815118e-01,  6.01990890e-01,  9.38398928e-01,
         1.05403772e+00,  1.70899078e+00,  2.42381396e+00,  3.25610375e+00]),
 array([[-2.09869386e-01,  1.28975702e-01, -9.70956830e-02,
          1.32792737e-01, -5.92617579e-02, -2.55792185e-01,
         -3.23010027e-01, -2.98719366e-01, -3.75133952e-17,
          7.55671318e-01,  3.67809805e-02, -7.60942241e-02,
          2.57525003e-02,  1.56127024e-01, -1.70476926e-01,
          1.32154295e-01],
        [-4.09905211e-01, -5.75141122e-02,  8.00666776e-02,
         -1.74789408e-01, -4.93180824e-01,  1.99774022e-01,
          7.50876193e-02,  4.16015878e-02, -7.01880935e-17,
         -4.95271890e-02, -2.77554124e-01, -2.30077181e-01,
          4.08877272e-01, -2.82922012e-01, -2.49072663e-01,
          2.43956109e-01],
        [-2.25106244e-01,  3.48579313e-0

**Pay attention!** Eigenvectors are the columns!

In [10]:
ff_eigens=la.eigh(ff_adj)

### Perron - Frobenius

In [11]:
np.all(ff_adj>=0)

True

The adjacency matrix is non negative (ok, that's easy)

In [12]:
np.unique(ff_eigens[0], return_counts=True)

(array([-2.69583872e+00, -2.06786891e+00, -1.87072241e+00, -1.19329397e+00,
        -8.69346719e-01, -7.65384963e-01, -5.76260635e-01, -2.02434814e-01,
        -6.53638385e-17,  2.57815118e-01,  6.01990890e-01,  9.38398928e-01,
         1.05403772e+00,  1.70899078e+00,  2.42381396e+00,  3.25610375e+00]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

$\Rightarrow$ All eigenvalues have multiplicity 1, thus the greatest eigenvalue has.

In [13]:
ff_eigens[0][-1]>-ff_eigens[0][0]

True

$\lambda>|\mu|,\,\forall\mu\in\sigma(A)$

In [14]:
ff_eigens[0][-1]

3.2561037454308566

In [15]:
ff_eigens[1][:,-1]

array([1.32154295e-01, 2.43956109e-01, 2.11705251e-01, 2.82800087e-01,
       2.59026167e-01, 7.49227077e-02, 2.89115599e-01, 8.87918880e-02,
       4.30308094e-01, 4.48134359e-02, 2.75730374e-01, 6.50521303e-18,
       3.41552644e-01, 1.45917196e-01, 3.55980448e-01, 3.25842301e-01])

In [16]:
ff_eigens[1][:, -1]>0

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

D'oh! So, what should we conclude?

In [17]:
ff_k=np.sum(ff_adj, axis=1)
print(ff_k)

[1 3 2 3 3 1 4 1 6 1 3 0 3 2 4 3]


A-HA! We got an isolated node!

In [18]:
where_iso=np.where(ff_k==0)[0][0]
families[where_iso]

'PUCCI\n'

None got married with the Pucci family! Actually this information was sort of hidden:

In [19]:
ff_eigens[0]

array([-2.69583872e+00, -2.06786891e+00, -1.87072241e+00, -1.19329397e+00,
       -8.69346719e-01, -7.65384963e-01, -5.76260635e-01, -2.02434814e-01,
       -6.53638385e-17,  2.57815118e-01,  6.01990890e-01,  9.38398928e-01,
        1.05403772e+00,  1.70899078e+00,  2.42381396e+00,  3.25610375e+00])

$10^{-17}$ is actually zero (due to the finite machine precision it is different from zero, but it means zero).

np.finfo returns the machine limits for a certain data type

In [20]:
np.finfo(np.float64).eps

2.220446049250313e-16

Let's try again, after removing the isolated node.

#### PF again

##### Pucci Mask and checks

In [21]:
pucci_mask=np.ones(len(ff_k), dtype='bool')

In [22]:
pucci_mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [23]:
pucci_mask[where_iso]=False

In [24]:
ff_new_families=families[pucci_mask]

In [25]:
ff_new_adj=ff_adj[pucci_mask]

In [26]:
ff_new_adj.shape

(15, 16)

In [27]:
ff_new_adj=ff_new_adj.T[pucci_mask]

In [28]:
ff_new_adj.shape

(15, 15)

Check!

In [29]:
ff_new_k=np.sum(ff_new_adj, axis=1)
print(ff_new_k)

[1 3 2 3 3 1 4 1 6 1 3 3 2 4 3]


In [30]:
np.all(np.dot(ff_new_adj,np.dot(ff_new_adj,np.dot(ff_new_adj,np.dot(ff_new_adj,np.dot(ff_new_adj,ff_new_adj)))))>0)

True

##### Let's try again

Ok, let's try again. The non-negativity is ok, since we just remove a row and a column filled with zeros. The same is true for the multiplicity (we removed the zero eigenvalue), but let's check.

In [31]:
np.all(ff_new_adj>=0)

True

In [32]:
ff_new_eigens=la.eigh(ff_new_adj)

In [33]:
ff_new_eigens[0]

array([-2.69583872, -2.06786891, -1.87072241, -1.19329397, -0.86934672,
       -0.76538496, -0.57626063, -0.20243481,  0.25781512,  0.60199089,
        0.93839893,  1.05403772,  1.70899078,  2.42381396,  3.25610375])

In [34]:
np.unique(ff_new_eigens[0], return_counts=True)

(array([-2.69583872, -2.06786891, -1.87072241, -1.19329397, -0.86934672,
        -0.76538496, -0.57626063, -0.20243481,  0.25781512,  0.60199089,
         0.93839893,  1.05403772,  1.70899078,  2.42381396,  3.25610375]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

Nothing should change even for the 2nd implication of the PF-theorem...

In [35]:
ff_new_eigens[0][-1]>-ff_new_eigens[0][0]

True

In [36]:
ff_new_eigens[1][:,-1]

array([0.13215429, 0.24395611, 0.21170525, 0.28280009, 0.25902617,
       0.07492271, 0.2891156 , 0.08879189, 0.43030809, 0.04481344,
       0.27573037, 0.34155264, 0.1459172 , 0.35598045, 0.3258423 ])

In [37]:
np.all(ff_new_eigens[1][:,-1]>0)

True

![](../Images/l4_turk.gif)

### Exercise: show that the Perron-Frobenius theorem is valid for the Zachary dataset

### Exercise: show that the (undirected) network represented by the edge ../Data/mysterious_dataset.txt is bipartite

In [38]:
roy=np.genfromtxt('../Data/mysterious_dataset.txt', dtype='i8')

In [39]:
len_matrix=np.max(roy)+1

In [40]:
matrix=np.zeros((len_matrix,len_matrix))
for r in roy:
    matrix[r[0], r[1]]=1
    matrix[r[1], r[0]]=1

In [41]:
np.all(matrix==matrix.T)

True

In [42]:
la.eigh(matrix)

(array([-6.74190812e+00, -4.38009830e+00, -2.44726084e+00, -2.11991083e+00,
        -1.98367917e+00, -1.73494800e+00, -1.53465722e+00, -1.23602800e+00,
        -1.01907939e+00, -9.80168425e-01, -7.07127953e-01, -6.25614086e-01,
        -3.99580655e-01, -8.50014503e-16, -7.25458711e-17,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  7.25458711e-17,  3.99580655e-01,
         6.25614086e-01,  7.07127953e-01,  9.80168425e-01,  1.01907939e+00,
         1.23602800e+00,  1.53465722e+00,  1.73494800e+00,  1.98367917e+00,
         2.11991083e+00,  2.44726084e+00,  4.38009830e+00,  6.74190812e+00]),
 array([[-0.23669268,  0.18540913, -0.24016878, ...,  0.24016878,
         -0.18540913,  0.23669268],
        [ 0.10036894, -0.13614462, -0.00529482, ..., -0.00529482,
         -0.13614462,  0.10036894],
        [-0.2186549 ,  0.20138354,  0.10429389, ..., -0.10429389,
         -0.20138354,  0.2186549 ],
        ...,
        [-0.09293883, -0.05959234, -0.26044164, ...,  0.26044164,
     

### Laplacian

Go back to the Medici family. Let's forget about Pucci and go to the Laplacian:

In [43]:
ff_new_L=np.diag(ff_new_k)-ff_new_adj

In [44]:
ff_new_L

array([[ 1,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  3,  0,  0,  0, -1, -1,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  2,  0, -1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  3,  0,  0, -1,  0,  0,  0, -1,  0,  0, -1,  0],
       [ 0,  0, -1,  0,  3,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0],
       [ 0, -1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0, -1,  0,  0,  4, -1,  0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0,  0],
       [-1, -1, -1,  0,  0,  0,  0,  0,  6,  0,  0, -1, -1,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, -1,  0,  0],
       [ 0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  3,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  3,  0, -1, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  2,  0,  0],
       [ 0,  0,  0, -1, -1,  0,  0,  0,  0,  0, -1, -1,  0,  4,  0],
       [ 0,  0,  0,  0,  0,  0, -1

In [45]:
ff_new_L_eigens=la.eigh(ff_new_L)

Some checks: 

In [46]:
np.all(ff_new_L_eigens[0]>=0)

True

In [47]:
ff_new_L_eigens[0][0]

1.048703484575426e-15

The smallest eigenvalue is actually zero.

In [48]:
ff_new_L_eigens[0][-1]

7.2682588444324345

In [49]:
np.max(ff_k)

6

Thus the greatest eigenvalue is greater than the greatest degree!

In [50]:
ff_new_L_eigens[0][1]

0.34592316467323053

The network is connected!

### Exercise: define a function that calculates the Laplacian

### Exercise: show that the Zachary network is connected

### Fiedler vector

Consider the Fiedler vector with $r=0$ and consider the relative partitions.

In [138]:
group_0=np.where(ff_new_L_eigens[1][:, 1]>=0)[0]
group_1=np.where(ff_new_L_eigens[1][:, 1]<0)[0]

In [139]:
print(ff_new_families[group_0])
print(ff_new_families[group_1])

['ALBIZZI\n' 'BARBADORI\n' 'BISCHERI\n' 'CASTELLAN\n' 'GINORI\n'
 'GUADAGNI\n' 'LAMBERTES\n' 'PERUZZI\n' 'RIDOLFI\n' 'STROZZI\n'
 'TORNABUON\n']
['ACCIAIUOL\n' 'MEDICI\n' 'PAZZI\n' 'SALVIATI\n']


#### What happens reapplying Fiedler partition to the previous partition?

##### group_0

In [140]:
ff_new_adj_0=ff_new_adj[group_0].T
ff_new_adj_0=ff_new_adj_0[group_0]

Check!

In [141]:
ff_new_adj_0.shape

(11, 11)

In [142]:
len(group_0)

11

In [143]:
np.all(ff_new_adj_0.T==ff_new_adj_0)

True

In [148]:
np.sum(ff_new_adj_0, axis=0)

array([2, 1, 3, 3, 1, 4, 1, 3, 2, 4, 2])

In [150]:
ff_new_families_0=ff_new_families[group_0]

In [144]:
def laplacianer(adj):
    deg=np.sum(adj, axis=1)
    return np.diag(deg)-adj

In [145]:
ff_new_L_0=laplacianer(ff_new_adj_0)

In [146]:
ff_new_L_0_eigens=la.eigh(ff_new_L_0)

In [147]:
ff_new_L_0_eigens[0]

array([1.38948074e-15, 2.73478502e-01, 5.86440285e-01, 8.69907821e-01,
       1.28707587e+00, 2.16569056e+00, 2.74778307e+00, 3.14380590e+00,
       4.29207459e+00, 5.11218075e+00, 5.52156266e+00])

This cluster is connected.

In [155]:
group_0_0=np.where(ff_new_L_0_eigens[1][:, 1]>=0)[0]
group_0_1=np.where(ff_new_L_0_eigens[1][:, 1]<0)[0]

In [160]:
ff_new_families_0

array(['ALBIZZI\n', 'BARBADORI\n', 'BISCHERI\n', 'CASTELLAN\n',
       'GINORI\n', 'GUADAGNI\n', 'LAMBERTES\n', 'PERUZZI\n', 'RIDOLFI\n',
       'STROZZI\n', 'TORNABUON\n'], dtype='<U10')

In [159]:
print(ff_new_families_0[group_0_0])
print(ff_new_families_0[group_0_1])

['ALBIZZI\n' 'GINORI\n' 'GUADAGNI\n' 'LAMBERTES\n' 'TORNABUON\n']
['BARBADORI\n' 'BISCHERI\n' 'CASTELLAN\n' 'PERUZZI\n' 'RIDOLFI\n'
 'STROZZI\n']


##### group_1

In [163]:
ff_new_adj_1=ff_new_adj[group_1].T
ff_new_adj_1=ff_new_adj_1[group_1]

Check!

In [164]:
ff_new_adj_1.shape

(4, 4)

In [165]:
len(group_1)

4

In [166]:
np.all(ff_new_adj_1.T==ff_new_adj_1)

True

In [167]:
np.sum(ff_new_adj_1, axis=0)

array([1, 2, 1, 2])

In [168]:
ff_new_families_1=ff_new_families[group_1]

In [169]:
ff_new_L_1=laplacianer(ff_new_adj_1)

In [170]:
ff_new_L_1_eigens=la.eigh(ff_new_L_1)

In [171]:
ff_new_L_1_eigens[0]

array([-9.74614466e-17,  5.85786438e-01,  2.00000000e+00,  3.41421356e+00])

This cluster is connected.

In [172]:
group_1_0=np.where(ff_new_L_1_eigens[1][:, 1]>=0)[0]
group_1_1=np.where(ff_new_L_1_eigens[1][:, 1]<0)[0]

In [173]:
ff_new_families_1

array(['ACCIAIUOL\n', 'MEDICI\n', 'PAZZI\n', 'SALVIATI\n'], dtype='<U10')

In [174]:
print(ff_new_families_1[group_1_0])
print(ff_new_families_1[group_1_1])

['ACCIAIUOL\n' 'MEDICI\n']
['PAZZI\n' 'SALVIATI\n']


Actually there is still space for reapplying the Fiedler vector algorithm!

### Exercise: check if there is the possibility to partition nodes further

### Exercise: define a function that get as input the adjacency matrix and returns the iterated partition in terms of the Fiedler vector
**Hint**: A *recursive* function is a function defined in terms of itself via self-referential expressions. Sometimes it helps...

### Exercise: apply the above function to the Zachary dataset